# Problem 0: Backoff language model (30%)

### We know that having unknown words in text is a problem for a language model. Any estimate of probability is difficult in such a scenario. 

### In class, we saw a simple way of smoothing probabilities by adding count 1 to every occuring ngram. While this can be a simple and effective technique we can do something a bit more clever. In this exercise we will implement two such techniques. 

### 1) to deal with unknown unigrams we will introduce a special `<unk>` token in our training data to represent rare tokens

### 2) for unknown bigrams we will use a technique called backoff. The idea is to "backoff" to a lower order n-gram estimate for the probability if the n-gram is unknown. For example the probability of an unknown bigram `w_1 w_2` can be estimated by looking at the unigram probability of `w_2`. 

In [1]:
%pylab inline

Populating the interactive namespace from numpy and matplotlib


In [3]:
import pandas as pd
import numpy as np
import re
from collections import Counter

wiki_df = pd.read_csv('./data/kdwd_r1k_articles.csv')

def get_tokens(text):
    return ['<s>'] + re.findall(r'\w+', text.lower()) + ['</s>']

train_sentences_list = ' '.join(wiki_df['intro_text'].iloc[:-100].tolist()).split('.')
test_sentences_list = ' '.join(wiki_df['intro_text'].iloc[-100:].tolist()).split('.')

In [4]:
train_sentences_list[:5]

['Apple Inc',
 ' is an American multinational technology company headquartered in Cupertino, California, that designs, develops, and sells consumer electronics, computer software, and online services',
 ' It is considered one of the Big Four tech companies along with Amazon, Google, and Facebook',
 " The company's hardware products include the iPhone smartphone, the iPad tablet computer, the Mac personal computer, the iPod portable media player, the Apple Watch smartwatch, the Apple TV digital media player, the AirPods wireless earbuds and the HomePod smart speaker",
 " Apple's software includes the macOS, iOS, iPadOS, watchOS, and tvOS operating systems, the iTunes media player, the Safari web browser, the Shazam acoustic fingerprint utility, and the iLife and iWork creativity and productivity suites, as well as professional applications like Final Cut Pro, Logic Pro, and Xcode"]

### First, let's build a basic 1-gram language model

In [9]:
train_token_list = [get_tokens(text) for text in train_sentences_list]

In [10]:
train_token_list[:3]

[['<s>', 'apple', 'inc', '</s>'],
 ['<s>',
  'is',
  'an',
  'american',
  'multinational',
  'technology',
  'company',
  'headquartered',
  'in',
  'cupertino',
  'california',
  'that',
  'designs',
  'develops',
  'and',
  'sells',
  'consumer',
  'electronics',
  'computer',
  'software',
  'and',
  'online',
  'services',
  '</s>'],
 ['<s>',
  'it',
  'is',
  'considered',
  'one',
  'of',
  'the',
  'big',
  'four',
  'tech',
  'companies',
  'along',
  'with',
  'amazon',
  'google',
  'and',
  'facebook',
  '</s>']]

In [11]:
train_token_list = [item for sublist in train_token_list for item in sublist]
train_token_list[:3]

['<s>', 'apple', 'inc']

In [12]:
unigram_counts = Counter()
# your code here
for word in train_token_list:
    unigram_counts[word] += 1
n_unigrams = np.sum([v for _, v in unigram_counts.items()])

In [8]:
assert(n_unigrams == 95491)

In [9]:
def get_unigram_token_prob(token):
    return unigram_counts[token] / n_unigrams

def get_text_prob_unigram(text):
    tokens = get_tokens(text)
    logp = 0
    for t in tokens:
        # code here
        logp += np.log(get_unigram_token_prob(t))
    return np.exp(logp)

In [10]:
assert(get_unigram_token_prob('apple').round(5) == 0.00046)
assert(get_text_prob_unigram('the company').round(9) == 2.455e-06)

### Note that we haven't yet introduced any smoothing, meaning, out-of-vocabulary words will have a probability of 0:

In [11]:
get_text_prob_unigram("onomatopoeia")

C:\Users\lhz98\Anaconda3\lib\site-packages\ipykernel_launcher.py:9: RuntimeWarning: divide by zero encountered in log
  if __name__ == '__main__':


0.0

### We have learned that we can simply add 1 to every word count to prevent this (ref: laplace smoothing). Another way however is to mark rare words within our training set as unknown words. The idea is that the model will then learn how to deal with unknown/rare words, to more correctly evaluate a test text.

### For this, let us first identify all unigrams that occur fewer or equal than k times. Let's use k=1 to start out with.

In [12]:
unigram_counts['apple']

44

In [13]:
rare_tokens = set()
# you loop code here
k = 1
for token in train_token_list:
    if unigram_counts[token] <= k:
        rare_tokens.add(token)

In [14]:
assert(len(rare_tokens) == 4859)

### Next, let's create a new counter `filtered_unigram_counts` where every token that appears in `rare_tokens` is recorded as the special token `<unk>`

In [15]:
filtered_unigram_counts = Counter()
for token_list in train_token_list:
# your code here
    if token_list in rare_tokens:
        filtered_unigram_counts['<unk>'] += 1
    else:
        filtered_unigram_counts[token_list] += 1

n_filtered_unigrams = np.sum([v for _, v in filtered_unigram_counts.items()])

In [16]:
assert(filtered_unigram_counts['<unk>'] == 4859)

### To use these new counts, let's modify our text probability function

In [17]:
def get_filtered_unigram_token_prob(token):
    return filtered_unigram_counts[token] / n_filtered_unigrams

def get_text_prob_filtered_unigram(text):
    tokens = [token if token in filtered_unigram_counts else '<unk>' for token in get_tokens(text)]# get tokens and convert to <unk> if needed
    logp = 0
    for t in tokens:
        logp += np.log(get_filtered_unigram_token_prob(t))
    return np.exp(logp)

In [18]:
assert(get_filtered_unigram_token_prob('apple').round(5) == 0.00046)
assert(get_text_prob_filtered_unigram('the company').round(9) == 2.455e-06)
assert(get_text_prob_filtered_unigram("onomatopoeia").round(5) == 0.00016)

### We can see that now unknown words actually have a probability higher than some of the rare words that we have already seen before like `apple`.

### The choice of count 1 to label words as `<unk>`was arbitrary. How could we tune is if we had more time?

In [19]:
# your text answer here
print('We can change the setup of rare words. For instance, we can set a smaller value for k. In such a way, the probability should be more reasonable.')

We can change the setup of rare words. For instance, we can set a smaller value for k. In such a way, the probability should be more reasonable.


### Let's expand our model to bigrams now. Make sure to check if each component in a bigram exists and label it as `<unk>` if needed.

In [20]:
filtered_bigram_counts = Counter()
for i in range(len(train_token_list)-1):
    # your loop and 'unk' conversion here
    t1 = train_token_list[i] if train_token_list[i] not in rare_tokens else '<unk>'
    t2 = train_token_list[i+1] if train_token_list[i+1] not in rare_tokens else '<unk>'
    filtered_bigram_counts[t1 + ' ' + t2] += 1

def get_filtered_bigram_token_prob(token1, token2):
    return filtered_bigram_counts[token1 + ' ' + token2] / filtered_unigram_counts[token1]
        
def get_text_prob_filtered_bigram(text):
    tokens = [token if token in filtered_unigram_counts else '<unk>' for token in get_tokens(text)]
    logp = 0
    for t1, t2 in zip(tokens[:-1], tokens[1:]):
        logp += np.log(get_filtered_bigram_token_prob(t1, t2))
    return np.exp(logp)

In [21]:
assert(get_text_prob_filtered_bigram('the company').round(5) == 0.00148)

### We correctly get a higher probabiliy for `the company`, now that we are respecting bigrams.
### However:

In [22]:
get_text_prob_filtered_bigram('company the')

C:\Users\lhz98\Anaconda3\lib\site-packages\ipykernel_launcher.py:15: RuntimeWarning: divide by zero encountered in log
  from ipykernel import kernelapp as app


0.0

### We can see that we still get 0 for unknown bigrams. Let's fix this via Backoff. To reiterate: the idea is to default to unigram probabilities if the bigram is unknown.

In [23]:
def get_backoff_bigram_token_prob(token1, token2):
    # check if bigram exists and if not return unigram token2 prob 
    if token1 + ' ' + token2 in filtered_bigram_counts:
        return get_filtered_bigram_token_prob(token1, token2)
    else:
        return get_filtered_unigram_token_prob(token2)

In [24]:
def get_text_prob_backoff_bigram(text):
    tokens = [token if token in filtered_unigram_counts else '<unk>' for token in get_tokens(text)]
    logp = 0
    for t1, t2 in zip(tokens[:-1], tokens[1:]):
        logp += np.log(get_backoff_bigram_token_prob(t1, t2))
    return np.exp(logp)

In [25]:
assert(get_text_prob_backoff_bigram('company the').round(8) == 1.1e-07)

### We can happily now estimate any input text we can think of with running into issues with 0.

### Let's see if this was all worth it. Let's evaluate perplexity.
### Specifically compare the perplexity of our filtered unigram model `get_filtered_unigram_token_prob` to our new and improved backoff bigram model `get_backoff_bigram_token_prob`

### Note: For easy comparison let's only evaluate `tokens[1:]` for both models such that even the first token can already form a correct bigram

In [26]:
def get_text_ppl_filtered_unigram(text):
    tokens = [token if token in filtered_unigram_counts else '<unk>' for token in get_tokens(text)]
    # your code
    n_tokens = len(tokens)
    logp = 0
    for t in tokens[1:-1]:
        logp += np.log(get_filtered_unigram_token_prob(t))
    return (1 / np.exp(logp))**(1 / n_tokens)

def get_text_ppl_backoff_bigram(text):
    tokens = [token if token in filtered_unigram_counts else '<unk>' for token in get_tokens(text)]
    # your code
    n_tokens = len(tokens)
    logp = 0
    for t1, t2 in zip(tokens, tokens[1:-1]):
        logp += np.log(get_backoff_bigram_token_prob(t1, t2))
    return (1 / np.exp(logp))**(1 / n_tokens)

In [27]:
ppl_list = []
for text in test_sentences_list:
    ppl_list.append(get_text_ppl_filtered_unigram(text))
model_unigram_ppl = np.mean(ppl_list)

In [28]:
ppl_list = []
for text in test_sentences_list:
    ppl_list.append(get_text_ppl_backoff_bigram(text))
model_bigram_ppl = np.mean(ppl_list)

In [29]:
assert(model_bigram_ppl < model_unigram_ppl)

### Seems like it worked very well. Try to find one or two examples of short strings that clearly show that our bigram model is better and why. (Short answer is OK here)

In [30]:
# your answer here

In [31]:
text = 'United States'

In [32]:
get_text_prob_backoff_bigram(text)

0.0004633571836538262

In [33]:
get_text_prob_filtered_unigram(text)

5.395621085297207e-08

In [34]:
text2 = 'States United'

In [35]:
get_text_prob_backoff_bigram(text2)

2.6115189766940493e-07

In [36]:
get_text_prob_filtered_unigram(text2)

5.395621085297207e-08

In [37]:
print('The string "United States" and its reverse clearly show that our bigram model is better since our bigram model takes order into consideration.')

The string "United States" and its reverse clearly show that our bigram model is better since our bigram model takes order into consideration.
